In [1]:
from scipy.cluster.vq import *
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
from imutils import paths
import pickle
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from skimage.feature import hog
from skimage.feature import local_binary_pattern
from skimage import filters
import pandas as pd

C:\Users\rep03\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
f = open('train.txt',"r", encoding='utf-8')
lines = f.readlines()
image_paths=[]  
image_classes=[]
for line in lines:
    curLine=line.strip().split(" ")
    image_paths.append(curLine[0]) 
    image_classes.append(curLine[-1])
time=1
allt=len(image_paths)
ims = []
print("loading images")
for image_path in image_paths:
    img  = cv2.imread(image_path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=cv2.resize(img,(64,64),interpolation=cv2.INTER_CUBIC)
    ims.append(img)
    print(str(time)+"/"+str(allt) +"  " + image_path, end="\r")
    time = time +1
print("\nDone!")

loading images
63325/63325  images/n02172182/n02172182_9821.JPEGGG
Done!


In [3]:
print(len(image_paths))
print(len(image_classes))

63325
63325


In [5]:
lbp = local_binary_pattern(img,  8, 1)
lbp_feature = []
time=1
allt=len(image_paths)
print("Calculating all the descriptors and keypoints...")
for im in ims:
    lbp=local_binary_pattern(im, 8, 1, 'default')
    max_bins = int(lbp.max() + 1)
    des, _ = np.histogram(lbp, normed=True, bins=max_bins, range=(0, max_bins))
    if str(des) != 'None':
        lbp_feature.append(des)       
        print(str(time) + "/" + str(allt), end="\r")
        time = time +1
    else:
        print(image_paths[time-1])
        del image_classes[time-1]
        del image_paths[time-1]
print("\nDone!")

Calculating all the descriptors and keypoints...
28/63325

C:\Users\rep03\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  if __name__ == '__main__':


63325/63325
Done!


In [6]:
lbp_feature

[array([0.07714844, 0.04858398, 0.00341797, 0.0065918 , 0.00830078,
        0.00146484, 0.00463867, 0.01220703, 0.00268555, 0.00146484,
        0.        , 0.00024414, 0.00537109, 0.00317383, 0.00976562,
        0.02392578, 0.05151367, 0.02612305, 0.00146484, 0.00317383,
        0.00268555, 0.00073242, 0.00219727, 0.00390625, 0.00732422,
        0.00317383, 0.        , 0.0012207 , 0.01196289, 0.00195312,
        0.02954102, 0.01904297, 0.00195312, 0.00097656, 0.00024414,
        0.00024414, 0.00024414, 0.        , 0.00048828, 0.00048828,
        0.        , 0.00024414, 0.        , 0.        , 0.00048828,
        0.        , 0.00146484, 0.00219727, 0.0078125 , 0.00195312,
        0.00024414, 0.00024414, 0.00024414, 0.00048828, 0.00097656,
        0.00024414, 0.00415039, 0.00073242, 0.        , 0.00024414,
        0.00512695, 0.00195312, 0.01171875, 0.00488281, 0.0090332 ,
        0.00146484, 0.00073242, 0.00146484, 0.0012207 , 0.        ,
        0.00097656, 0.00048828, 0.00073242, 0.  

In [7]:
pca = PCA(n_components=128)
lbp_feature=pca.fit_transform(lbp_feature)

In [18]:
lbpclf = LinearSVC()
lbpclf.fit(lbp_feature,image_classes)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [9]:
g = open('val.txt',"r", encoding='utf-8')
lines_val = g.readlines()
image_paths_val=[]  
image_classes_val=[]
for line1 in lines_val:
    curLine_val=line1.strip().split(" ")
    image_paths_val.append(curLine_val[0]) 
    image_classes_val.append(curLine_val[-1])
time1=1
allt1=len(image_paths_val)
ims_val = []
print("loading images")
for image_path1 in image_paths_val:
    img1  = cv2.imread(image_path1)
    img1=cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    img1=cv2.resize(img1,(64,64),interpolation=cv2.INTER_CUBIC)
    ims_val.append(img1)
    print(str(time1)+"/"+str(allt1) +"  " + image_path1, end="\r")
    time1 = time1 +1
print("\nDone!")

loading images
450/450  images/n02172182/n02172182_9982.JPEG
Done!


In [10]:
print(len(image_paths_val))
print(len(image_classes_val))

450
450


In [11]:
lbp_val = local_binary_pattern(img1,  8, 1)
lbp_feature_val = []
time1=1
allt1=len(image_paths_val)
print("Calculating all the descriptors and keypoints...")
for im1 in ims_val:
    lbp1=local_binary_pattern(im1, 8, 1, 'default')
    max_bins = int(lbp1.max() + 1)
    des, _ = np.histogram(lbp1, normed=True, bins=max_bins, range=(0, max_bins))
    if str(des) != 'None':
        lbp_feature_val.append(des)    
        print(str(time1) + "/" + str(allt1), end="\r")
        time1 = time1 +1
    else:
        print(image_paths_val[time1-1])
        del image_classes_val[time1-1]
        del image_paths_val[time1-1]
print("\nDone!")

Calculating all the descriptors and keypoints...


C:\Users\rep03\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  if __name__ == '__main__':


450/450
Done!


In [12]:
pca = PCA(n_components=128)
lbp_feature_val=pca.fit_transform(lbp_feature_val)

In [13]:
h = open('test.txt',"r", encoding='utf-8')
lines_test = h.readlines()
image_paths_test=[]  
image_classes_test=[]
for line2 in lines_test:
    curLine_test=line2.strip().split(" ")
    image_paths_test.append(curLine_test[0]) 
    image_classes_test.append(curLine_test[-1])
time2=1
allt2=len(image_paths_test)
ims_test = []
print("loading images")
for image_path2 in image_paths_test:
    img2  = cv2.imread(image_path2)
    img2=cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    img2=cv2.resize(img2,(64,64),interpolation=cv2.INTER_CUBIC)
    ims_test.append(img2)
    print(str(time2)+"/"+str(allt2) +"  " + image_path2, end="\r")
    time2 = time2 +1
print("\nDone!")

loading images
450/450  images/n02172182/n02172182_981.JPEGG
Done!


In [14]:
print(len(image_paths_test))
print(len(image_classes_test))

450
450


In [15]:
lbp_test = local_binary_pattern(img2,  8, 1)
lbp_feature_test = []
time2=1
allt2=len(image_paths_test)
print("Calculating all the descriptors and keypoints...")
for im2 in ims_test:
    lbp2=local_binary_pattern(im2, 8, 1, 'default')
    max_bins = int(lbp2.max() + 1)
    des, _ = np.histogram(lbp2, normed=True, bins=max_bins, range=(0, max_bins))
    if str(des) != 'None':
        lbp_feature_test.append(des)     
        print(str(time2) + "/" + str(allt2), end="\r")
        time2 = time2 +1
    else:
        print(image_paths_test[time2-1])
        del image_classes_test[time2-1]
        del image_paths_test[time2-1]
print("\nDone!")

Calculating all the descriptors and keypoints...


C:\Users\rep03\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  if __name__ == '__main__':


450/450
Done!


In [16]:
pca = PCA(n_components=128)
lbp_feature_test=pca.fit_transform(lbp_feature_test)

In [19]:
predict=lbpclf.predict(lbp_feature_test)

In [20]:
print(accuracy_score(image_classes_test,predict))

0.044444444444444446
